Dask

In [ ]:
#from dask_jobqueue import SLURMCluster
#cluster = SLURMCluster(queue='analysis',walltime='00:20:00',memory='96GB',cores=8,project='gfdl_f')

In [ ]:
from dask.distributed import Client
client = Client('140.208.147.176:45978')

In [ ]:
client

### SST OM4p25

In [ ]:
import cartopy
import cmocean
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.colorbar
import xarray as xr
import numpy

#### xarray doesn't load data from disk except when needed, just deals with the metadata. objects refer to actual values

In [ ]:
OM4p25 = xr.open_zarr('/work/Nkeh.Boh/SST/OM4p25-100yr/Zarrfile/OM4p25.zarr') #zarr 100 series files

In [ ]:
W25 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/WOA25.zarr')

In [ ]:
wtemp_mean = W25['ptemp'].mean(axis=(0,1))

In [ ]:
#OM4p25.chunk({'time':1200,'xh':1440,'yh':1080})

In [ ]:
%%time
temp_mean = OM4p25['tos'].mean(dim=['time'] ).load() #.

In [ ]:
temp_mean.shape

## Plots

In [ ]:
fig = plt.figure(figsize=(8,5))
vmin,vmax,ci,cmap = -2.25,2.25,.5,plt.cm.RdYlBu_r
axes = []
cilev = numpy.arange(vmin-ci,vmax+ci*2,ci)
norm = matplotlib.colors.BoundaryNorm(boundaries=cilev, ncolors=cmap.N)

q = temp_mean - wtemp_mean #

ax = plt.axes(projection=cartopy.crs.Robinson(central_longitude=-155))

im = ax.pcolormesh(q, vmin=vmin, vmax=vmax)
#stats(ax, a25, q, 'OMp25')
ax.coastlines()
axes.append(ax)
plt.title('a) OM4p25')

In [ ]:
G25 = xr.open_zarr('/work/Nkeh.Boh/zarrfiles_old/G25')
G5 = xr.open_zarr('/work/Nkeh.Boh/zarrfiles_old/G5')
M25 = xr.open_zarr('/work/Nkeh.Boh/zarrfiles_old/M25')

W25 = xr.open_zarr('/work/Nkeh.Boh/zarrfiles_old/W25') #Get zarr series file for this (W25)

xq25 = G25.variables['x'][:][::2,::2]
yq25 = G25.variables['y'][:][::2,::2]
a25 = G25.variables['area'][:]; a25 = a25[::2,::2]+a25[1::2,1::2]+a25[1::2,::2]+a25[1::2,::2]
m25 = M25.variables['mask'][:]; a25 = a25*m25


def stats(ax, area, anomaly, label):
    mn = (anomaly*area).sum()/area.sum()
    sd = numpy.sqrt( ((anomaly-mn)**2*area).sum()/area.sum() )
    rms = numpy.sqrt( (anomaly**2*area).sum()/area.sum() )
    qmn, qmx = anomaly.min(), anomaly.max()
    print(label, 'mean =', mn, 'sd =', sd, 'rms =', rms, 'min =', qmn, 'max =', qmx )
    bb = ax.get_position()
    plt.gcf().text(bb.x0,bb.y1+.01,'mean=%.3f$^\circ$C'%mn, horizontalalignment='left')
    plt.gcf().text(bb.x1,bb.y1+.01,'rms=%.3f$^\circ$C'%rms, horizontalalignment='right')

    
OM4p25_av = OM4p25['tos'].mean(dim='time')

W25_av = W25['ptemp'][0].mean(axis=0)

fig = plt.figure(figsize=(8,5))
vmin,vmax,ci,cmap = -2.25,2.25,.5,plt.cm.RdYlBu_r
axes = []
cilev = numpy.arange(vmin-ci,vmax+ci*2,ci)
norm = matplotlib.colors.BoundaryNorm(boundaries=cilev, ncolors=cmap.N)

q = OM4p25_av - W25_av  #

ax = plt.axes(projection=cartopy.crs.Robinson(central_longitude=-155))

im = ax.pcolormesh(xq25, yq25, q, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, vmin=vmin, vmax=vmax)
stats(ax, a25, q, 'OMp25')
ax.coastlines()
axes.append(ax)
plt.title('a) OM4p25')

